# Profundizando en las operaciones con Strings


Una de las fortalezas de Python es su relativa facilidad para manejar y manipular cadenas de texto.

Pandas se basa en esto para proporcionar un conjunto completo de operaciones de texto *vectorizadas*, que son una pieza esencial para extraer información de los textos que pueda ser analizada posteriormente, ya que es prácticamente imposible trabajar con cadenas de texto en bruto.

En esta sección, analizaremos algunas de las operaciones con strings de Pandas, y luego veremos cómo usarlas para limpiar parcialmente un conjunto de datos muy desordenado de recetas recopiladas de Internet.

## Introducción a las operaciones con strings de Pandas

Hemos visto cómo herramientas como NumPy y Pandas generalizan las operaciones aritméticas para poder trabajar con conjuntos de datos, como por ejemplo:

In [1]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Este fenómeno es a lo que nos referimos cuando hablamos de vectorización de operaciones. De este modo, se simplifica la sintaxis a emplear para operar con matrices de datos, pues ya no tendremos que preocuparnos por el tamaño o la forma de la matriz, sino por la operación que queremos que se realice.

Para matrices de cadenas, NumPy no proporciona un acceso tan simple, por lo que no es posible realizar estas operaciones más que iterando sobre cada uno de los elementos, lo que reduce su eficiencia y dificulta su uso:

In [4]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Además, dado que no está pensado para ello, solo nos permite trabajar con algunos datos. Esto hace que en casos más complejos, como datos faltantes, no sea capaz de ejecutar las operaciones.
Por ejemplo:

In [5]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas incluye características para abordar tanto esta necesidad de operaciones de cadenas vectorizadas como para manejar correctamente los datos faltantes a través del atributo ``str`` de los objetos ``Series`` de Pandas que contienen cadenas.

Veámoslo con un ejemplo:

In [6]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

Ahora, si quisiéramos unificar el formato de los nombres para poder compararlos entre ellos y que queden bonitos, podríamos optar por convertirlos de forma que todos tuvieran su primera letra mayúscula y el resto en minúsculas.

Con NumPy, hemos visto que esto no era posible por los valores nulos. Sin embargo, con Pandas es tan simple como:

In [7]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

Podemos hacer uso de las ventajas de los notebooks pulsando el ``Tabulador`` trasn escribir nuestro objeto ``Series`` seguido de ``.str.``, donde nos aparecerán todas las opciones posibles. Pruébalo en la sigueinte celda, simplemente ponte al final del ``.`` y pulsa la tecla ``Tabulador``:

In [ ]:
names.str.

Además, los métodos ``str`` también nos sirven con los índices:

In [20]:
names.index = ['aaa', 'bBb', None, 'ddD', 'eee']
print(names.index)
names.index.str.capitalize()

Index(['aaa', 'bBb', None, 'ddD', 'eee'], dtype='object')


Index(['Aaa', 'Bbb', None, 'Ddd', 'Eee'], dtype='object')

In [21]:
names.index = names.index.str.capitalize()
names.str.capitalize()

Aaa    Peter
Bbb     Paul
NaN     None
Ddd     Mary
Eee    Guido
dtype: object

## Tabla resumen de los métodos de Pandas para strings

La mayoría de estos métodos se parecen a los vistos para las cadenas de texto de Python, por lo que nos resultará bastante intuitivo. Pero no sólo dispondremos de estos métodso, sino que habrá otros añadidos con los que les podremos sacar más jugo a las cadenas de texto.

En esta sección, comenzaremos por los métodos comunes, pasaremos a las expresiones regulares, y terminaremos por otros métodos propios de los ``Series``.

In [24]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Métodos similares a Python

Casi todos los métodos de Python para tratar strings han sido replicados por Pandas de forma vectorizada. A continuación, se recoge una lista con estos métodos, accesibles con el prefijo ``.str.``, como hemos visto anteriormente:

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()``    | ``lower()``      | ``translate()``  | ``islower()``    | 
|``ljust()``  | ``upper()``      | ``startswith()`` | ``isupper()``    | 
|``rjust()``  | ``find()``       | ``endswith()``   | ``isnumeric()``  | 
|``center()`` | ``rfind()``      | ``isalnum()``    | ``isdecimal()``  | 
|``zfill()``  | ``index()``      | ``isalpha()``    | ``split()``      | 
|``strip()``  | ``rindex()``     | ``isdigit()``    | ``rsplit()``     | 
|``rstrip()`` | ``capitalize()`` | ``isspace()``    | ``partition()``  | 
|``lstrip()`` |  ``swapcase()``  |  ``istitle()``   | ``rpartition()`` |


Fíjate que, dado que estamos trabajando con operaciones vectorizadas sobre ``Series`` de strings, el resultado no será otro ``Series``.

De strings (objects):

In [26]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Numérico:

In [27]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

Booleano:

In [28]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Incluso otros tipos particulares, como listas:

In [29]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

### Métodos usando expresiones regulares

Ademñas, hay varios métodos que nos permitirán trabajar con expresiones regulares, que son patrones que nos permitirán encontrar coincidencias en el texto, como buscar todos los números que aparezcan en un string, o si una cadena empieza con un caracter en concreto. Estos métodos están basados en el módulo ``re``:

| Method | Description |
|--------|-------------|
| ``match()`` | Llama a ``re.match()`` para cada elemento, devolviendo un booleano. |
| ``extract()`` | Llama a ``re.match()`` para cada elemento, devolviendo los grupos de cadenas coincidentes.|
| ``findall()`` | Llama a ``re.findall()`` para cada elemento. |
| ``replace()`` | Reemplaza las ocurrencias del patrón por otro string. |
| ``contains()`` | Llama a ``re.search()`` para cada elemento, devolviendo un booleano. |
| ``count()`` | Cuenta las ocurrencias del patrón. |
| ``split()``   | Equivalente a ``str.split()``, pero acepta expresiones regulares. |
| ``rsplit()`` | Equivalente a ``str.rsplit()``, pero acepta expresiones regulares. |

Si bien es cierto que esto es una herramienta muy potente, meternos a fondo con las expresiones regulares se sale de lo que buscamos en este curso, por lo que dejaremos algún ejemplo sencillo y, en el caso de necesitar algo más complejo, recurriremos a la documentación. Si quieres un sitio web donde está muy bien explicado, te dejo aquí el [enlace](https://www.programaenpython.com/miscelanea/expresiones-regulares/).

Con estos métodos, podemos realizar una gran cantidad de operaciones interesantes.

Por ejemplo, podemos extraer el nombre de cada uno pidiendo un grupo contiguo de caracteres al comienzo de cada elemento:

In [30]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

In [40]:
monte.str.extract('^([A-Za-z]+)')

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


También podríamos extraer letras sueltas, como todas las ``a`` de cada nombre:

In [161]:
monte_a = monte.str.findall(r'a')
monte_a

0    [a, a, a, a]
1              []
2             [a]
3              []
4              []
5          [a, a]
dtype: object

### Para pensar...

En vista de esto, ¿se te ocurre cómo podríamos extraer el número de ``a`` que tiene cada nombre? Podríamos usar el método ``apply`` para recorrer uno a uno los diferentes registros y hacer un recuento de los valores de cada lista:

0    4
1    0
2    1
3    0
4    0
5    2
dtype: int64

Podríamos complicarlo un poco, añadiendo ciertas condiciones, como quedarnos con las vocales de los nombres:

In [169]:
monte_a = monte.str.findall(r'[aeiou]')
monte_a

0       [a, a, a, a]
1       [o, e, e, e]
2       [e, i, i, a]
3             [i, e]
4          [e, o, e]
5    [i, a, e, a, i]
dtype: object

O podríamos hacer algo más complicado, como buscar todso los nombres que empiecen y terminen con una consonante, haciendo uso de los comandos especiales de inicio de string (``^``) o fin de string (``$``), además del comdín que fincionará como cualqueir caracter (``*``) o el punto (``.``) que nos permitirá repetir tantas veces como sean el caracter que le indiquemos a continuación. Cbe destacar, que el acento circunflejo (``^``) al comienzo de algo definido entre corchetes (``[]``) significa negación:

In [46]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

Como puedes ver, la posibilidad de utilizar expresiones regulares en nuestros tratamientos de datos abren un sinfín de posibilidades, auqneu ya te digo que no nos centraremos en ello como tal, sino que si necesitamos extraer algún patrón en concreto, nos dirigiremos a la documentación, ya que es un tema que puede llegar a ser muy complejo y variante, en función de lo que queramos obtener.

### Métodos misceláneos

Finalmente, también tenemos otros métodos que no podemos cuadrar en nignuna de los 2 puntos anteriores, lo que denominaremos métodso misceláneos, que nos posibilitan realizar otras operaciones:

| Method | Description |
|--------|-------------|
| ``get()`` | Devuelve cada elemento según su índice |
| ``slice()`` | Slice de cada elemento |
| ``slice_replace()`` | Reemplaza cada slice en cada elemento con el valor pasado |
| ``cat()``      | Concatena strings |
| ``repeat()`` | Repite valores |
| ``normalize()`` | Devuelve el Unicode de un string |
| ``pad()`` | Añade un espacio en blanco a la izquierda, derecha, o ambos lados de los strings |
| ``wrap()`` | Divida las cadenas largas en líneas con una longitud menor que el ancho determinado |
| ``join()`` | Une las cadenas de cada elemento de un ``Series`` con el separador indicado |
| ``get_dummies()`` | Extrae las variables dummy como un ``DataFrame`` (hace grupos con los valores) |

Las operaciones ``get()`` y ``slice()``, permiten el acceso vectorizado a elementos de cada matriz.

Por ejemplo, podemos obtener un slice de los 3 primeros caracteres de cada array usando ``str.slice(0, 3)``.

*Nota:* teniendo en mente cómo funcionaba el slicing de Python, esto sería equivalente a usar ``df.str[0:3]``

In [51]:
monte.str.slice(0, 3)

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

In [52]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

Indexar mediante ``df.str.get(i)`` y ``df.str[i]`` es bastante similar.

Los métodos ``get()`` y ``slice()`` se pueden combinar para acceder a los elementos de los arrays devueltos por ``split()``.

For example, para extraer el último nombre de cada entrada, podemos hacer lo sigueinte:

In [53]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

Otro método que requiere un poco más de atención es el método `` get_dummies () ``.

Lo usaremos cuando nuestros datos tienen una columna que contiene algún tipo de indicador codificado.

Por ejemplo, podríamos tener un conjunto de datos que contenga información en forma de códigos (variables categóricas), como A = "nacido en España," B = "nacido en el extranjero," C = "le gusta la montaña," D = "le gustan los videojuegos":

In [54]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


El métodp ``get_dummies()`` nos permite separar rápidamente en variables de un ``DataFrame`` estos indicadores puestos en forma de string:

In [55]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Con esto ganamos mucha flexibilidad a la hora de estudiar y procesar nuestros conjuntos de datos con strings.


## Ejercicios:

Léete el fichero "FIFA20_mod.csv" y realiza algunas operaciones:
1. Créate una columna con el nombre del "club" en mayúsculas. Comprueba que realmente has cambiado el nombre a mayúsculas en otra columna utilizando el método adecuado (equivalente en Python)
2. Créate otra columna nueva con el "team_position" en minúsculas y comprueba, como has hecho antes, que realmente está en minúsculas.
3. Si te fijas, los nombres "long_name" han sufrido modificaciones en este fichero. Utiliza alguna de las funciones comunes entre Pandas y Python que hemos visto para ponerlo todo de modo que la primera letra de cada palabra sea mayúscula, y el resto minúsculas (hay un método concreto para eso).
4. Resulta que hay otro cambio más respecto al fichero original, y es que la columna 'overall' no es de tipo entero, ya que se han colado un punto y unos espacios delante de su valor. Guárdate los valores de la columna en una varaible (que será de tipo ``Series``) y quítale el punto con el método ``replace``. A continuación, al objeto ``Series`` que devuelve esta función, elimínale los espacios de la izquierda (que son los que estaban entre el punto y el número). Para ello busca la función en la tabla de funciones de Pandas similares a Python. Tras ello, comprueba que son dígitos (con una de las funciones de esa misma tabla). El resultado deberá ser un ``Series`` con valores a ``True`` o ``False`` en función de que sean dígitos o no. Recuerda que para acceder a estas funciones debemos poner el ``.str.`` entre nuestro objeto ``Series`` y la función.
5. Tras hacer el ejercicio anterior, te darás cuenta de que no todos son dígitos (en realidad, ninguno lo será). Muestra por pantalla el primer elemento a ver si te da una pista de lo que está ocurriendo. Cuando identifiques qué tiene de más que le hace no ser un dígito, elimínalo con la función correspondiente y vuelve a comprobar si ya está solucionado el problema con la función que has utilizado antes.
6. Volvamos sobre los nombres de los futbolistas. Utiliza una expresión regular para obtener un objeto ``Series`` con las vocales de los nombres ("long_name") de cada uno de los futbolistas. Haz el recuento de vocales en otro objeto ``Series``.
7. Utiliza una de las funcinoes que hemos visto para crear otro ``DataFrame`` con diferentes columnas que indiquen el "team_position" de cada jugador.

In [189]:
df = pd.read_csv("FIFA20_mod.csv")
df

,short_name,long_name,dob,club,overall,potential,value_eur,wage_eur,team_position,preferred_foot,influencer
0,L. Messi,lionel andrés messi cuccittini,1987-06-24,FC Barcelona,. 94,94,95500000,565000,RW,Left,True
1,Cristiano Ronaldo,CRISTIANO RONALDO DOS SANTOS AVEIRO,1985-02-05,Juventus,. 93,93,58500000,405000,LW,Right,True
2,Neymar Jr,NEYMAR DA SILVA SANTOS JUNIOR,1992-02-05,Paris Saint-Germain,. 92,92,105500000,290000,CAM,Right,True
3,J. Oblak,Jan oblak,1993-01-07,Atlético Madrid,. 91,93,77500000,125000,GK,Right,False
4,E. Hazard,Eden hazard,1991-01-07,Real Madrid,. 91,91,90000000,470000,LW,Right,True
...,...,...,...,...,...,...,...,...,...,...,...
18273,Shao Shuai,邵帅,1997-03-10,Beijing Renhe FC,. 48,56,40000,2000,RES,Right,False
18274,Xiao Mingjie,MINGJIE XIAO,1997-01-01,Shanghai SIPG FC,. 48,56,40000,2000,SUB,Right,False
18275,Zhang Wei,张威,2000-05-16,Hebei China Fortune FC,. 48,56,40000,1000,SUB,Right,False
18276,Wang Haijian,汪海健,2000-08-02,Shanghai Greenland Shenhua FC,. 48,54,40000,1000,SUB,Right,False


In [198]:
# 1. 
df['club_mayus'] = df['club'].str.upper()
df['club_mayus'].str.isupper()
# df['club_mayus'].str.isupper().all()

0        True
1        True
2        True
3        True
4        True
         ... 
18273    True
18274    True
18275    True
18276    True
18277    True
Name: club_mayus, Length: 18278, dtype: bool

In [199]:
# 2. 
df['club_minus'] = df['club'].str.lower()
df['club_minus'].str.islower()
# df['club_minus'].str.islower().all()

0        True
1        True
2        True
3        True
4        True
         ... 
18273    True
18274    True
18275    True
18276    True
18277    True
Name: club_minus, Length: 18278, dtype: bool

In [201]:
# 3. 
df['long_name'] = df['long_name'].str.title()
df['long_name']

0             Lionel Andrés Messi Cuccittini
1        Cristiano Ronaldo Dos Santos Aveiro
2              Neymar Da Silva Santos Junior
3                                  Jan Oblak
4                                Eden Hazard
                        ...                 
18273                                     邵帅
18274                           Mingjie Xiao
18275                                     张威
18276                                    汪海健
18277                                    潘喜明
Name: long_name, Length: 18278, dtype: object

In [203]:
# 4. 
df['overall'] = df['overall'].str.replace('.', '').str.lstrip()
df['overall'].str.isdigit()

0        False
1        False
2        False
3        False
4        False
         ...  
18273    False
18274    False
18275    False
18276    False
18277    False
Name: overall, Length: 18278, dtype: bool

In [204]:
# 5. 
df['overall'] = df['overall'].str.rstrip()
df['overall'].str.isdigit()

0        True
1        True
2        True
3        True
4        True
         ... 
18273    True
18274    True
18275    True
18276    True
18277    True
Name: overall, Length: 18278, dtype: bool

In [209]:
# 6. 
ser6 = df['long_name'].str.findall('[aeiou]')
ser6.apply(lambda x: len(x))

0         9
1        13
2        10
3         2
4         3
         ..
18273     0
18274     6
18275     0
18276     0
18277     0
Name: long_name, Length: 18278, dtype: int64

In [211]:
# 7. 
df_pos = df['team_position'].str.get_dummies()
df_pos['jugador'] = df['long_name']
df_pos

,CAM,CB,CDM,CF,CM,GK,LAM,LB,LCB,LCM,...,RDM,RES,RF,RM,RS,RW,RWB,ST,SUB,jugador
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Lionel Andrés Messi Cuccittini
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Cristiano Ronaldo Dos Santos Aveiro
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neymar Da Silva Santos Junior
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Jan Oblak
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Eden Hazard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18273,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,邵帅
18274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Mingjie Xiao
18275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,张威
18276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,汪海健


### Ejericio 2: Combinando conceptos

1. Utiliza una de las funciones de Python que tiene Pandas para el tratamimento de strings para obtener todos aquellos clubs que tienen "FC" en el nombre. Obtén una lista con los equipos únicos.
2. Realiza el mismo ejercicio pero usando la función ``findall`` y la expresión regular que sea conveniente. Esto te devolverá un ``Series`` con listas, a partir de él, obtén otro ``Series`` booleano de la misma longitud que te diga si la cadena está vacía o no. Tras ello, utiliza este ``Series`` para filtrar la columna 'club' original, y quédate con aquellos que tengan 'FC' en el nombre. La variable final no deberá tener duplicados (si uno aparece 2 veces, quédate con 1, no deseches los 2)
3. Repite el ejercicio 2 pero para aquellos equipos que empiecen por 'FC'
4. Repite el paso 2 para aquellos equipos que empiecen por 'FC'

In [ ]:
# 1. 
df[df['club'].str.contains("FC")]['club'].unique()

In [212]:
# 2. 
df[df['club'].str.findall("FC").apply(lambda x: len(x) != 0)]['club'].unique()

array(['FC Barcelona', 'FC Bayern München', 'FC Porto',
       'Dalian YiFang FC', 'Guangzhou Evergrande Taobao FC',
       'Shanghai SIPG FC', 'Jiangsu Suning FC', 'Los Angeles FC',
       'Sevilla FC', 'Shanghai Greenland Shenhua FC', 'PFC CSKA Moscow',
       'Beijing Sinobo Guoan FC', '1. FC Köln', 'Guangzhou R&F FC',
       'FC Girondins de Bordeaux', 'FC Nantes', 'FC Schalke 04',
       'Hebei China Fortune FC', 'Toronto FC', 'Beijing Renhe FC',
       'Girona FC', 'Shenzhen FC', 'FC Augsburg', 'FC København',
       'Stade Rennais FC', 'Royal Antwerp FC', 'Tianjin TEDA FC',
       'Tianjin Quanjian FC', 'Seattle Sounders FC',
       'Shandong Luneng TaiShan FC', 'FC Red Bull Salzburg',
       '1. FC Union Berlin', 'Chongqing Dangdai Lifan FC SWM Team',
       'FC Utrecht', 'FC Basel 1893', 'Rangers FC', 'Western United FC',
       'New York City FC', 'Minnesota United FC', 'FC Groningen',
       'Junior FC', 'Ettifaq FC', 'Avaí FC', '1. FC Nürnberg', 'FC Metz',
       'FC Midtjy

In [213]:
# 3. 
df[df['club'].str.findall("^FC").apply(lambda x: len(x) != 0)]['club'].unique()

array(['FC Barcelona', 'FC Bayern München', 'FC Porto',
       'FC Girondins de Bordeaux', 'FC Nantes', 'FC Schalke 04',
       'FC Augsburg', 'FC København', 'FC Red Bull Salzburg',
       'FC Utrecht', 'FC Basel 1893', 'FC Groningen', 'FC Metz',
       'FC Midtjylland', 'FC Sion', 'FC Tokyo', 'FCSB (Steaua)',
       'FC Lorient', 'FC Twente', 'FC St. Pauli', 'FC Dallas',
       'FC Cincinnati', 'FC Zürich', 'FC Viitorul', 'FC Lugano',
       'FC Paços de Ferreira', 'FC Seoul', 'FC Nordsjælland', 'FC Thun',
       'FC Juárez', 'FC Ingolstadt 04', 'FC Emmen', 'FC Erzgebirge Aue',
       'FC Luzern', 'FC Sochaux-Montbéliard', 'FC Botoşani',
       'FC St. Gallen', 'FC Admira Wacker Mödling', 'FC Hansa Rostock',
       'FC Hermannstadt', 'FC Würzburger Kickers', 'FC Chambly Oise',
       'FC Voluntari', 'FC Carl Zeiss Jena'], dtype=object)

In [214]:
# 4. 
df[df['club'].str.findall("FC$").apply(lambda x: len(x) != 0)]['club'].unique()

array(['Dalian YiFang FC', 'Guangzhou Evergrande Taobao FC',
       'Shanghai SIPG FC', 'Jiangsu Suning FC', 'Los Angeles FC',
       'Sevilla FC', 'Shanghai Greenland Shenhua FC',
       'Beijing Sinobo Guoan FC', 'Guangzhou R&F FC',
       'Hebei China Fortune FC', 'Toronto FC', 'Beijing Renhe FC',
       'Girona FC', 'Shenzhen FC', 'Stade Rennais FC', 'Royal Antwerp FC',
       'Tianjin TEDA FC', 'Tianjin Quanjian FC', 'Seattle Sounders FC',
       'Shandong Luneng TaiShan FC', 'Rangers FC', 'Western United FC',
       'New York City FC', 'Minnesota United FC', 'Junior FC',
       'Ettifaq FC', 'Avaí FC', 'Moreirense FC', 'Boavista FC',
       'Sydney FC', 'Al Ain FC', 'Henan Jianye FC', 'Panathinaikos FC',
       'Puebla FC', 'Daegu FC', 'Rio Ave FC', 'Millonarios FC',
       'Paris FC', 'Servette FC', 'Valenciennes FC', 'Ulsan Hyundai FC',
       'Gil Vicente FC', 'Randers FC', 'Vancouver Whitecaps FC',
       'Gyeongnam FC', 'Melbourne City FC', 'Damac FC', 'Envigado FC',
       